In [32]:
import numpy as np

def silt_and_clay_columns(df):
    # Handle silt and clay columns if present
    if {'silt', 'clay'}.issubset(df.columns):
        df['silt_plus_clay'] = df[['silt', 'clay']].sum(axis=1, skipna=True)
        df.drop(columns=['silt', 'clay'], inplace=True)
    
    return df

def om_and_bd_columns(df):
    # Calculate organic matter metrics if orgc present
    if 'orgc' in df.columns:
        # Use vectorized operations for better performance
        organic_matter = 1.724 * df['orgc']
        df = df.assign(
            organic_matter=organic_matter,
            bulk_density=1.62 - 0.06 * organic_matter
        )

    return df

def clean_dataset(df):
    # drop columns with 70% or more missing values
    df = df.dropna(thresh=0.3 * len(df), axis=1)
    # remove outliers using z-score
    # df = df[(df.select_dtypes(include=[np.number]).apply(lambda x: x.abs() < 3).all(axis=1))]
    # remove negative values // cannot do this as we have negative values in the data for bulk density
    # df = df[df.select_dtypes(include=[np.number]).ge(0).all(axis=1)]
    # replace invalid dates
    # df['date'] = df.apply(lambda row: replace_invalid_dates(row['date']), axis=1)

    return df


In [ ]:
import pandas as pd

# load the raw soil dataset
df = pd.read_csv("../../outputs/Mexico_wosis_merged.csv")
# add silt and clay columns
df = silt_and_clay_columns(df)
# add organic matter and bulk density columns
df = om_and_bd_columns(df)

# save the cleaned dataset
df

In [ ]:
# group obervations by latitude and longitude to check for duplicates
duplicates = df.groupby(['latitude', 'longitude']).size().reset_index(name='count').copy()
# show only duplicates

sorted_duplicates = duplicates[duplicates['count'] > 1].sort_values(by='count', ascending=False)
sorted_duplicates

In [ ]:
import folium
from folium.plugins import HeatMap

grouped_data = df.groupby(['latitude', 'longitude']).size().reset_index(name='count')

map_h = folium.Map(
    location=[grouped_data['latitude'].mean(), grouped_data['longitude'].mean()],
    tiles="OpenStreetMap",
    zoom_start=4,
    min_zoom=4,
    max_zoom=6
)

gradient = {0.2: 'blue', 0.4: 'green', 0.6: 'yellow', 0.8: 'red'}
HeatMap(grouped_data,
        name="Soil Observations",
        radius=9, 
        blur=8,
        min_opacity=0.2,
        ).add_to(map_h)

map_h

In [ ]:
# heatmap of upper and lower depths
import matplotlib.pyplot as plt
import seaborn as sns

# Create a 2D histogram (heatmap) of upper vs lower depths
plt.figure(figsize=(10, 8))
plt.hist2d(df['upper_depth'], df['lower_depth'], bins=50, cmap='viridis')
plt.colorbar(label='Count')

# Set titles and labels
plt.title('Heatmap of Upper vs Lower Depths')
plt.xlabel('Upper Depth (cm)')
plt.ylabel('Lower Depth (cm)')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

# Convert date column to datetime and handle invalid dates
df['date'] = pd.to_datetime(df['date'], errors='coerce')
# create quarters
df['quarter'] = df['date'].dt.quarter
df['year'] = df['date'].dt.year

# Get distribution of years
plt.figure(figsize=(12, 6))
df['date'].dt.year.hist(bins=30)
plt.title('Distribution of Soil Measurements by Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

In [38]:
# plot the spatial distribution of the data for the selected year
# Load Mexico shapefile
mexico = gpd.read_file("D:/tierra/data/natural_earth/ne_110m_admin_0_countries.shp")
mexico = mexico[mexico['ADMIN'] == 'Mexico']


In [ ]:
# Filter for a specific year (e.g. 2000)
YEAR = 2004
df_year = df[df['year'] == YEAR]

print(f"Number of measurements in year {YEAR}: {len(df_year)}")

fig, axes = plt.subplots(2, 2, figsize=(8, 8))
axes = axes.ravel()

for quarter in range(1, 5):
    ax = axes[quarter-1]
    # Get data for this quarter
    quarter_data = df_year[df_year['quarter'] == quarter]
    
    # Plot Mexico boundaries
    mexico.plot(ax=ax, color='lightgrey', edgecolor='black')
    # Plot points
    ax.scatter(quarter_data['longitude'], quarter_data['latitude'], 
              c=quarter_data['date'], cmap='viridis', alpha=0.6)
    ax.set_title(f'Soil Measurements in Q{quarter} 2000 (n={len(quarter_data)})')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

plt.suptitle(f'Spatial Distribution of Soil Measurements in {YEAR} by Quarter')
plt.tight_layout()
plt.show()